In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# 1. CARGA DE DATOS
df = pd.read_csv('./../data/supply_chain_dataset1.csv')

In [4]:
df

,Date,SKU_ID,Warehouse_ID,Supplier_ID,Region,Units_Sold,Inventory_Level,Supplier_Lead_Time_Days,Reorder_Point,Order_Quantity,Unit_Cost,Unit_Price,Promotion_Flag,Stockout_Flag,Demand_Forecast
0,2024-01-01,SKU_1,WH_1,SUP_8,West,10,592,14,379,0,13.95,20.48,0,0,8.52
1,2024-01-02,SKU_1,WH_1,SUP_8,West,17,575,14,379,0,13.95,20.48,0,0,18.63
2,2024-01-03,SKU_1,WH_1,SUP_8,North,35,540,14,379,0,13.95,20.48,1,0,39.62
3,2024-01-04,SKU_1,WH_1,SUP_8,South,24,516,14,379,0,13.95,20.48,0,0,19.43
4,2024-01-05,SKU_1,WH_1,SUP_8,West,21,495,14,379,0,13.95,20.48,0,0,18.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91245,2024-12-26,SKU_50,WH_5,SUP_10,South,17,352,7,283,0,6.65,9.60,0,0,19.82
91246,2024-12-27,SKU_50,WH_5,SUP_10,South,21,331,7,283,0,6.65,9.60,0,0,27.96
91247,2024-12-28,SKU_50,WH_5,SUP_10,East,17,314,7,283,0,6.65,9.60,0,0,22.13
91248,2024-12-29,SKU_50,WH_5,SUP_10,South,24,290,7,283,0,6.65,9.60,1,0,24.11


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91250 entries, 0 to 91249
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Date                     91250 non-null  object 
 1   SKU_ID                   91250 non-null  object 
 2   Warehouse_ID             91250 non-null  object 
 3   Supplier_ID              91250 non-null  object 
 4   Region                   91250 non-null  object 
 5   Units_Sold               91250 non-null  int64  
 6   Inventory_Level          91250 non-null  int64  
 7   Supplier_Lead_Time_Days  91250 non-null  int64  
 8   Reorder_Point            91250 non-null  int64  
 9   Order_Quantity           91250 non-null  int64  
 10  Unit_Cost                91250 non-null  float64
 11  Unit_Price               91250 non-null  float64
 12  Promotion_Flag           91250 non-null  int64  
 13  Stockout_Flag            91250 non-null  int64  
 14  Demand_Forecast       

In [ ]:
# 2. TRANSFORMACIÓN (Limpieza y Tipos de Datos)
# Convertimos la fecha al formato correcto
df['Date'] = pd.to_datetime(df['Date'], dayfirst=False)

In [7]:
df.head(2)

,Date,SKU_ID,Warehouse_ID,Supplier_ID,Region,Units_Sold,Inventory_Level,Supplier_Lead_Time_Days,Reorder_Point,Order_Quantity,Unit_Cost,Unit_Price,Promotion_Flag,Stockout_Flag,Demand_Forecast
0,2024-01-01,SKU_1,WH_1,SUP_8,West,10,592,14,379,0,13.95,20.48,0,0,8.52
1,2024-01-02,SKU_1,WH_1,SUP_8,West,17,575,14,379,0,13.95,20.48,0,0,18.63


In [8]:
# 3. FEATURE ENGINEERING (Métricas de Negocio)
# Calculamos Ingresos y Ganancia por fila
df['Revenue'] = df['Units_Sold'] * df['Unit_Price']
df['Profit'] = (df['Unit_Price'] - df['Unit_Cost']) * df['Units_Sold']

In [9]:
df.head()

,Date,SKU_ID,Warehouse_ID,Supplier_ID,Region,Units_Sold,Inventory_Level,Supplier_Lead_Time_Days,Reorder_Point,Order_Quantity,Unit_Cost,Unit_Price,Promotion_Flag,Stockout_Flag,Demand_Forecast,Revenue,Profit
0,2024-01-01,SKU_1,WH_1,SUP_8,West,10,592,14,379,0,13.95,20.48,0,0,8.52,204.80,65.30
1,2024-01-02,SKU_1,WH_1,SUP_8,West,17,575,14,379,0,13.95,20.48,0,0,18.63,348.16,111.01
2,2024-01-03,SKU_1,WH_1,SUP_8,North,35,540,14,379,0,13.95,20.48,1,0,39.62,716.80,228.55
3,2024-01-04,SKU_1,WH_1,SUP_8,South,24,516,14,379,0,13.95,20.48,0,0,19.43,491.52,156.72
4,2024-01-05,SKU_1,WH_1,SUP_8,West,21,495,14,379,0,13.95,20.48,0,0,18.70,430.08,137.13


In [10]:
# Cálculo del Error de Pronóstico (Análisis de precisión)
df['Forecast_Error'] = df['Units_Sold'] - df['Demand_Forecast']
df['ABS_Forecast_Error'] = df['Forecast_Error'].abs()

In [13]:
df.head()

,Date,SKU_ID,Warehouse_ID,Supplier_ID,Region,Units_Sold,Inventory_Level,Supplier_Lead_Time_Days,Reorder_Point,Order_Quantity,Unit_Cost,Unit_Price,Promotion_Flag,Stockout_Flag,Demand_Forecast,Revenue,Profit,Forecast_Error,ABS_Forecast_Error
0,2024-01-01,SKU_1,WH_1,SUP_8,West,10,592,14,379,0,13.95,20.48,0,0,8.52,204.80,65.30,1.48,1.48
1,2024-01-02,SKU_1,WH_1,SUP_8,West,17,575,14,379,0,13.95,20.48,0,0,18.63,348.16,111.01,-1.63,1.63
2,2024-01-03,SKU_1,WH_1,SUP_8,North,35,540,14,379,0,13.95,20.48,1,0,39.62,716.80,228.55,-4.62,4.62
3,2024-01-04,SKU_1,WH_1,SUP_8,South,24,516,14,379,0,13.95,20.48,0,0,19.43,491.52,156.72,4.57,4.57
4,2024-01-05,SKU_1,WH_1,SUP_8,West,21,495,14,379,0,13.95,20.48,0,0,18.70,430.08,137.13,2.30,2.30


In [24]:
# Lógica de Salud de Inventario (Semaforización técnica)
def check_stock(row):
    if row['Inventory_Level'] <= row['Reorder_Point']:
        return 'Critical'
    elif row['Inventory_Level'] <= (row['Reorder_Point'] * 1.2):
        return 'Warning'
    else:
        return 'Healthy'

df['Inventory_Health'] = df.apply(check_stock, axis=1)

In [25]:
df

,Date,SKU_ID,Warehouse_ID,Supplier_ID,Region,Units_Sold,Inventory_Level,Supplier_Lead_Time_Days,Reorder_Point,Order_Quantity,Unit_Cost,Unit_Price,Promotion_Flag,Stockout_Flag,Demand_Forecast,Revenue,Profit,Forecast_Error,ABS_Forecast_Error,Inventory_Health
0,2024-01-01,SKU_1,WH_1,SUP_8,West,10,592,14,379,0,13.95,20.48,0,0,8.52,204.80,65.30,1.48,1.48,Healthy
1,2024-01-02,SKU_1,WH_1,SUP_8,West,17,575,14,379,0,13.95,20.48,0,0,18.63,348.16,111.01,-1.63,1.63,Healthy
2,2024-01-03,SKU_1,WH_1,SUP_8,North,35,540,14,379,0,13.95,20.48,1,0,39.62,716.80,228.55,-4.62,4.62,Healthy
3,2024-01-04,SKU_1,WH_1,SUP_8,South,24,516,14,379,0,13.95,20.48,0,0,19.43,491.52,156.72,4.57,4.57,Healthy
4,2024-01-05,SKU_1,WH_1,SUP_8,West,21,495,14,379,0,13.95,20.48,0,0,18.70,430.08,137.13,2.30,2.30,Healthy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91245,2024-12-26,SKU_50,WH_5,SUP_10,South,17,352,7,283,0,6.65,9.60,0,0,19.82,163.20,50.15,-2.82,2.82,Healthy
91246,2024-12-27,SKU_50,WH_5,SUP_10,South,21,331,7,283,0,6.65,9.60,0,0,27.96,201.60,61.95,-6.96,6.96,Warning
91247,2024-12-28,SKU_50,WH_5,SUP_10,East,17,314,7,283,0,6.65,9.60,0,0,22.13,163.20,50.15,-5.13,5.13,Warning
91248,2024-12-29,SKU_50,WH_5,SUP_10,South,24,290,7,283,0,6.65,9.60,1,0,24.11,230.40,70.80,-0.11,0.11,Warning


In [30]:
df.columns

Index(['Date', 'SKU_ID', 'Warehouse_ID', 'Supplier_ID', 'Region', 'Units_Sold',
       'Inventory_Level', 'Supplier_Lead_Time_Days', 'Reorder_Point',
       'Order_Quantity', 'Unit_Cost', 'Unit_Price', 'Promotion_Flag',
       'Stockout_Flag', 'Demand_Forecast', 'Revenue', 'Profit',
       'Forecast_Error', 'ABS_Forecast_Error', 'Inventory_Health'],
      dtype='object')

In [31]:
# 4. CARGA A MYSQL
# Configura tus credenciales locales
USER = 'root'
PASSWORD = input('contraseña: ')
HOST = 'localhost'
DATABASE = 'sch1_proyect'

engine = create_engine(f"mysql+mysqlconnector://{USER}:{PASSWORD}@{HOST}/{DATABASE}")

try:
    # Creamos la tabla y subimos la data enriquecida
    df.to_sql('processed_inventory_1',con=engine, if_exists='append', index=False)
    print("¡Data procesada y cargada exitosamente en MySQL!")
except Exception as e:
    print(f"Error al conectar con MySQL: {e}")

# Mostrar vista previa de lo que subimos
print(df.head())

¡Data procesada y cargada exitosamente en MySQL!
        Date SKU_ID Warehouse_ID Supplier_ID Region  Units_Sold  \
0 2024-01-01  SKU_1         WH_1       SUP_8   West          10   
1 2024-01-02  SKU_1         WH_1       SUP_8   West          17   
2 2024-01-03  SKU_1         WH_1       SUP_8  North          35   
3 2024-01-04  SKU_1         WH_1       SUP_8  South          24   
4 2024-01-05  SKU_1         WH_1       SUP_8   West          21   

   Inventory_Level  Supplier_Lead_Time_Days  Reorder_Point  Order_Quantity  \
0              592                       14            379               0   
1              575                       14            379               0   
2              540                       14            379               0   
3              516                       14            379               0   
4              495                       14            379               0   

   Unit_Cost  Unit_Price  Promotion_Flag  Stockout_Flag  Demand_Forecast  \
0  